In [2]:
!pip install -U spacy
import spacy
nlp = spacy.load("en_core_web_sm")



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)
########################################################################
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Baseline_MUSS_corrupt
########################################################################
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

########################################################################


# !ssh-add ~/.ssh/id_rsa
!git clone https://github.com/facebookresearch/muss.git
%cd muss/
!pip install -e .  # Install package
# !python -m spacy download en_core_web_md  # Install required spacy models
########################################################################
# import pandas as pd

# import spacy
import re
import math
import pickle
from itertools import product
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os
########################################################################
from pathlib import Path
def yield_lines(filepath, gzipped=False, n_lines=None):
    filepath = Path(filepath)
    open_function = open
    if gzipped or filepath.name.endswith('.gz'):
        open_function = gzip.open
        
    with open_function(filepath, 'rt') as f:
        for i, l in enumerate(f):
            if n_lines is not None and i >= n_lines:
                break
            yield l.rstrip('\n')

def read_lines(filepath, gzipped=False):
    return list(yield_lines(filepath, gzipped=gzipped))

########################################################################

pd.set_option('display.max_rows', 50)

nlp = spacy.load("en_core_web_sm")
    

Mounted at /content/drive
/content/drive/MyDrive/Baseline_MUSS_corrupt
Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.
fatal: destination path 'muss' already exists and is not an empty directory.
/content/drive/MyDrive/Baseline_MUSS_corrupt/muss
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Baseline_MUSS_corrupt/muss
  Cloning https://github.com/feralvam/easse.git to /tmp/pip-install-f8f1bv2p/easse_769c8fef4b1a43bf9bd72544694e8fb2
  Running command git clone -q https://github.com/feralvam/easse.git /tmp/pip-install-f8f1bv2p/easse_769c8fef4b1a43bf9bd72544694e8fb2
  Cloning https://github.com/kpu/kenlm.git to /tmp/pip-install-f8f1bv2p/kenlm_69a5982686e046ee90a8527d4c82b379
  Running command git clone -q https://github.com/kpu/kenlm.git /tmp/pip-install-f8f1bv2p/kenlm_69a5982686e046ee90a8527d4c82b379
     |███████████████████████████████

Load wiki-large

In [ ]:
stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]


In [ ]:
DATA_DIR = '/content/drive/MyDrive/muss/resources/datasets/'

asset_valid_complex_dir =  DATA_DIR + f'token_asset_0803/valid.complex'
asset_test_complex_dir = DATA_DIR + f'token_asset_0803/test.complex'

asset_valid_simple_dir = DATA_DIR + f'token_asset_simple_NER_0810/valid.complex'
asset_test_simple_dir = DATA_DIR + f'token_asset_simple_NER_0810/test.complex'

asset_valid_all_no_random_dir = DATA_DIR + f'asset_simple_all_words_no_random_0825/valid.complex'
asset_test_all_no_random_dir = DATA_DIR + f'asset_simple_all_words_no_random_0825/test.complex'

asset_valid_all_random_dir = DATA_DIR + f'asset_simple_all_words_random_0825/valid.complex'
asset_test_all_random_dir = DATA_DIR + f'asset_simple_all_words_random_0825/test.complex'

complex_model_data_test_dir_list=[asset_test_complex_dir,asset_test_simple_dir]
complex_model_data_test_list=['asset.complex.test','asset.simple.test']

test_data_dir_list,test_data_list = [],[]

tmp_data_dir_list = []
tmp_data_list = []

for phase in ['test']:
  for i in range(7):
    path_name = 'asset_'+phase+'_'+str(i)+'_simple_dir'
    globals()[path_name] = DATA_DIR + f'asset_simple_'+str(i)+'NE_0825/'+phase+'.complex'

    tmp_data_dir_list.append(globals()[path_name])
    tmp_data_list.append('asset.'+str(i)+'.simple.'+phase)


test_data_dir_list.extend(tmp_data_dir_list)
test_data_list.extend(tmp_data_list)

test_data_dir_list
test_data_list

['asset.0.simple.test',
 'asset.1.simple.test',
 'asset.2.simple.test',
 'asset.3.simple.test',
 'asset.4.simple.test',
 'asset.5.simple.test',
 'asset.6.simple.test']

In [ ]:
test_data_dir_list.extend([
            asset_test_simple_dir,
             asset_test_complex_dir,
             asset_test_all_no_random_dir,
             asset_test_all_random_dir
             ]
)

test_data_list.extend([
            'asset.simple.test',
             'asset.complex.test',
             'asset.norandom.test',
             'asset.random.test',
             ]
)

In [ ]:
PHASES = ['train','valid', 'test']
LANGUAGES = ['complex','simple',]
data_names = []

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_detokenized/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language
  data_names.append(data_name)

  with open(data_dir+phase+'.'+language) as f:
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()


In [ ]:
# %ls '/content/drive/MyDrive/Baseline_MUSS/muss/resources/datasets/'
# test_complex

In [ ]:
# dataframe.iloc[0][0]

In [ ]:
# dataframe.iloc[0][1]

In [ ]:
# preparing for data
from muss.resources.prepare import *
b = prepare_wikilarge()
a = prepare_wikilarge_detokenized()


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/content/drive/MyDrive/Baseline_MUSS/muss/muss/utils/helpers.py", line 384, in __enter__
    sys.settrace(lambda *args, **keys: None)



WikiLarge
Done.
Removing empty directory wikilarge_detokenized
Creating "wikilarge_detokenized"...
WikiLarge
Done.


In [ ]:
data_names

['train_complex',
 'train_simple',
 'valid_complex',
 'valid_simple',
 'test_complex',
 'test_simple']

In [ ]:
train_data = {'complex':train_complex,'simple':train_simple}
train_df = pd.DataFrame(train_data)
# train_df.iloc[0][0]

In [ ]:
train_df

,complex,simple
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...
...,...,...
296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...
296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou..."
296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.
296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t..."


In [ ]:
stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]
cap_stop_words_list = [x.title() for x in stop_words_list]
total_stop_words_list = []
total_stop_words_list.extend(cap_stop_words_list)
total_stop_words_list.extend(stop_words_list)
len(total_stop_words_list)

1568

In [ ]:
# remove stop words
for index,file_name in enumerate(data_names):
    
    data = globals()[file_name]
    
    for count, sentence in enumerate(data):

        first_word = sentence.split(' ')[0]

        if first_word.lower() in stop_words_list:
            # delete it
            data[count] = sentence[len(first_word)+1:]
            

In [ ]:
train_complex[0]

'is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.'

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
data_names

['train_complex',
 'train_simple',
 'valid_complex',
 'valid_simple',
 'test_complex',
 'test_simple']

In [ ]:
with open('/content/drive/MyDrive/Baseline_MUSS/muss/quantative/NER_list_wikilarge_0810_complex.pickle', 'rb') as fp:
  data_NER = pickle.load(fp)

complex_data_NER = data_NER[0]
simple_data_NER = data_NER[1]


In [ ]:
# print([len(i) for i in data_NER])
complex_data_NER[0:10]
len(data_NER[0])

296402

In [ ]:
# define stop words list
cap_stop_words_list = [x.title() for x in stop_words_list]
total_stop_words_list = []
total_stop_words_list.extend(cap_stop_words_list)
total_stop_words_list.extend(stop_words_list)
print(len(total_stop_words_list))

####################################################
def create_clean_data_NER(data):
  clean_data_NER = []
  for i in data:
    temp_list = []
    temp_list.append([x for x in i if x not in cap_stop_words_list])
    clean_data_NER += temp_list
  return clean_data_NER
####################################################
### clean the data in an organized manner ***
### train/valid/test ### 
### complex, simple, intersection ### 

for i,data_set in enumerate(data_NER):
  print(i)
  # for j in range(2):
  data_NER[i] = create_clean_data_NER(data_set)
  print(len(data_set))
################################################

clean_complex_data_NER = data_NER[0::2]
clean_simple_data_NER = data_NER[1::2]


1568
0
296402
1
296402
2
992
3
992
4
359
5
359


In [ ]:

a = [1,2,3,4,5,6]
a[1::2]

[2, 4, 6]

In [ ]:
# clean_complex_data_NER = []
# for i in complex_data_NER:
#   temp_list = []
#   # print(i)
#   temp_list.append([x for x in i if x not in cap_stop_words_list])
#   # print(temp_list)
#   clean_complex_data_NER += temp_list

# clean_simple_data_NER = []
# for i in simple_data_NER:
#   temp_list = []
#   # print(i)
#   temp_list.append([x for x in i if x not in cap_stop_words_list])
#   # print(temp_list)
#   clean_simple_data_NER += temp_list

In [ ]:
clean_complex_data_NER[0:10]

[[['James Edward Austen', 'as late as 1814', 'Anna', '1809', '11'],
  ['Psalms of Thomas', 'Säve-Söderberg', 'Mandaean', 'Mani'],
  ['a period each year', 'Persephone', 'Hermes', 'Hades', 'six'],
  ['Cogeneration'],
  ['Switzerland', 'Romandie', 'Geneva', 'second', 'Zürich', 'French'],
  ['ten years later', 'Suzuka', 'Japan', 'F1'],
  ['its first month', 'August 1909'],
  ['the United Kingdom of Great Britain',
   'Hawaiian Independence Day',
   'The Kingdom of Hawai',
   'July Monarchy France',
   "Ka La Ku'oko '",
   'Ireland'],
  ['Oxford English Dictionary',
   'Oxford University Press',
   'Grim Reaper',
   'Scythe',
   '1933'],
  ['Toshihide Saito', 'Japanese'],
  ['American English', '25', '50', '75', '12', '4', '8'],
  ['Germany Stadtpfeifer', 'Italy Pifferi', 'Stadspijpers', 'Holland'],
  ['the 16th century', 'Germany in the', 'Mephistopheles', 'Faust', 'Plays'],
  ['Mary Stuart Masterson',
   'Mary-Louise Parker',
   'Carol Sobieski',
   'Jessica Tandy',
   'Fannie Flagg',
  

In [ ]:

# for i,j in enumerate(clean_complex_data_NER):
#   clean_complex_data_NER[i] = j[0]
# # clean_complex_data_NER[0]

# for i,j in enumerate(clean_simple_data_NER):
#   clean_simple_data_NER[i] = j[0]


In [ ]:
with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/NER_clean_list_wikilarge_0810_complex.plk', 'wb') as fp:
    pickle.dump(clean_complex_data_NER, fp)   

with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/NER_clean_list_wikilarge_0810_simple.plk', 'wb') as fp:
    pickle.dump(clean_simple_data_NER, fp)

In [ ]:
%ls '/content/drive/MyDrive/Baseline_MUSS/muss/'

CODE_OF_CONDUCT.md  muss_system_outputs/  scripts/
CONTRIBUTING.md     qualitative/          setup.cfg
experiments/        quantative/           setup.py
LICENSE             README.md             wikilarge_detokenized/
muss/               requirements.txt
muss.egg-info/      resources/


Data Cleaning 

In [ ]:
with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_clean_list_wikilarge_0810_complex.plk', 'rb') as fp:
    clean_complex_data_NER = pickle.load(fp)

with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_clean_list_wikilarge_0810_simple.plk', 'rb') as fp:
    clean_simple_data_NER = pickle.load(fp)



In [ ]:
len(clean_simple_data_NER)

3

In [ ]:
# clean_complex_data_NER


### Only intersection 
# len(clean_complex_data_NER)
cooc_list_NER = [] 
for i in range(3):
  cooc_list_NER.append([])
  for index,words in enumerate(clean_complex_data_NER[i]):
    if list(set(words)&set(clean_simple_data_NER[i][index])) == []:
      # print('empty')
      a = list()
      cooc_list_NER[i].append(a)
    else: 
      cooc_list_NER[i] += [list(set(words)&set(clean_simple_data_NER[i][index]))]
# print(cooc_list_NER[0][0:10])

In [ ]:
# clean_complex_data_NER[0:10]
# print(cooc_list_NER[0][0:10])

In [ ]:
# clean_simple_data_NER[0:10]


In [ ]:
### Create intersection and dump it 
with open ('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/NER_cooc_0823.pickle', 'wb') as fp:
    pickle.dump(cooc_list_NER,fp)

In [ ]:

with open ('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/NER_cooc_0823.pickle', 'rb') as fp:
    intersect_NE = pickle.load(fp)


creating training file 

In [ ]:
# load original sentence

data_names = []

PHASES = ['train', 'valid', 'test']
LANGUAGES = ['complex','simple']

data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_detokenized/'

for phase, language in product(PHASES, LANGUAGES):
  # print(phase, language)
  data_name = phase + '_' + language + '_ori'
  data_names.append(data_name)
  with open(data_dir+phase+'.'+language) as f:
        
    exec("%s = %s" % (data_name,f.read().splitlines()))

    f.close()
data_names

# train_complex_ori

['train_complex_ori',
 'train_simple_ori',
 'valid_complex_ori',
 'valid_simple_ori',
 'test_complex_ori',
 'test_simple_ori']

In [ ]:
# PHASES
for index,phase in enumerate(PHASES):
  data_tuples = list(zip(globals()[phase+'_complex_ori'],globals()[phase+'_simple_ori'],clean_complex_data_NER[index],clean_simple_data_NER[index],intersect_NE[index]))
  df_name = phase + '_df'
  print(df_name)
  # print(data_tuples[0:10])
  globals()[df_name] = pd.DataFrame(data_tuples, columns=['complex','simple','complex_NER','simple_NER','co_occur_NER'])
  # file_name = data_name.replace('_','.')

train_df
valid_df
test_df


In [ ]:
valid_df

# a = pd.DataFrame(data_tuples,columns=['complex','simple','complex_NER','simple_NER','co_occur_NER'])
# a

,complex,simple,complex_NER,simple_NER,co_occur_NER
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Hornja Å uÅ 3\/4 ica...",[the European Charter for Regional or Minority...,"[Saxony, Upper Sorbian]"
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,"[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...","[American Treasures, Library of Congress in Wa..."
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,[the Nobel Prize The Sveriges Riksbank Prize i...,[Sveriges Riksbank Prize in Economic Sciences ...,[]
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[]
4,"Helicopter rotor, the rotary wing (s) of a rot...","ROTOR, a former radar project in the UK follow...",[Helicopter],"[the Second World War, ROTOR, UK]",[]
...,...,...,...,...,...
987,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...","[more than 60 years, 1859]","[more than 60 years, 1859]","[more than 60 years, 1859]"
988,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...",[Southeast Asia]
989,"Raspberries, blackberries, and dewberries are ...",These plants have sharp thorns {prickles} like...,[Raspberries],[],[]
990,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,"[the Byzantine Empire, Eastern Orthodox, the p...","[the Byzantine Empire, Eastern Orthodox, Greec...","[Greece, Eastern Orthodox, Russia, the Byzanti..."


In [ ]:
train_df.drop(train_df[train_df.simple.map(lambda x: '.' not in x)].index, inplace=True)
# train_df.reset_index(inplace=True)

c=0
for index, row in train_df.iterrows():
  if all([x.lower() not in row['complex'].lower() for x in row['simple'].split(' ')]):
    # print(row['complex'])
    # print(row['simple'])
    # print()
    train_df.drop(index, inplace=True)
    c+=1

train_df

,complex,simple,complex_NER,simple_NER,co_occur_NER
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, as late as 1814, Anna, 1...","[James Edward, Anna Austen, 1814]",[]
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Mandaean, Mani, Säve-Söderb..."
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, Persephone, Hermes, Hades...","[Underworld, Persephone, Demeter, Hades]","[Persephone, Hades]"
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[]
...,...,...,...,...,...
296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[France, Saint-Maixant]"
296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[]
296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe, one]","[one, Boulogne-Billancourt, Europe]"


In [ ]:
c


5422

In [ ]:
valid_df.drop(valid_df[valid_df.simple.map(lambda x: '.' not in x)].index, inplace=True)
# valid_df.reset_index(inplace=True)

In [ ]:
c=0
for index, row in valid_df.iterrows():
  if all([x.lower() not in row['complex'].lower() for x in row['simple'].split(' ')]):
    valid_df.drop(index, inplace=True)
    c+=1

In [ ]:
c
valid_df.reset_index(inplace=True)

In [ ]:
train_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_train_df.pkl")
valid_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_valid_df.pkl")
test_df.to_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_test_df.pkl")

Create Training set

```
# This is formatted as code
```



In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_train_df.pkl")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_valid_df.pkl")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/final_wiki_test_df.pkl")

target_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/wikilarge_final_complex_0821'
# os.makedirs(target_dir)

In [ ]:
train_df

,complex,simple,complex_NER,simple_NER,co_occur_NER
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, as late as 1814, Anna, 1...","[James Edward, Anna Austen, 1814]",[]
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Mandaean, Mani, Säve-Söderb..."
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, Persephone, Hermes, Hades...","[Underworld, Persephone, Demeter, Hades]","[Persephone, Hades]"
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[]
...,...,...,...,...,...
296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[France, Saint-Maixant]"
296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[]
296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe, one]","[one, Boulogne-Billancourt, Europe]"


In [ ]:
train_df2 = train_df.reset_index(inplace=True)
train_df2

In [ ]:
data_names = ['train_complex', 'valid_complex', 'test_complex']
df_list = ['train_df','valid_df','test_df']

In [ ]:
### Adding additional Token for training sets 

for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        NER = row.complex_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))


  f.close()

0
 <NEXT_NE> James Edward Austen <NEXT_NE> as late as 1814 <NEXT_NE> Anna <NEXT_NE> 1809 <NEXT_NE> 11 <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> CMYK <NEXT_NE> RYB <NEXT_NE> Red <SENT_START> Red is also one of the subtractive primary colors of RYB color space but not CMYK color space.
20000
 <NEXT_NE> Reebok <SENT_START> During ceremonies, the players covered their Reebok logos.
30000
 <NEXT_NE> Beauharnais in Malmaison <NEXT_NE> first <NEXT_NE> Jos <SENT_START> The first known rose garden was planted by empress JosÃ phine de Beauharnais in Malmaison.
40000
 <NEXT_NE> more than one <SENT_START> Most elements have more than one possible oxidation state.
50000
 <NEXT_NE> Royal Opera House <NEXT_NE> Benjamin Britten <NEXT_NE> 1 December 1951 <NEXT_NE> E. M. Forster <NEXT_NE> Covent Garden <NEXT_

### NE are from simple sentence

In [ ]:
target_dir = '/content/drive/MyDrive/Baseline_MUSS/muss/resources/datasets/wikilarge_final_simple_0821'
# os.makedirs(target_dir)

In [ ]:
data_names = ['train_simple', 'valid_simple', 'test_simple']
df_list = ['train_df','valid_df','test_df']

In [ ]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        # print(sentence)
        NER = row.simple_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))

  f.close()


0
 <NEXT_NE> James Edward <NEXT_NE> Anna Austen <NEXT_NE> 1814 <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> Prime Minister of Lebanon <NEXT_NE> 1992 <NEXT_NE> 1998 <NEXT_NE> 2000 <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
20000
 <NEXT_NE> Varanasi <NEXT_NE> Lucknow <NEXT_NE> Kanpur <NEXT_NE> Agra <SENT_START> Domestic airports are Lucknow, GorakhpurAgra, Kanpur Allahabad and Varanasi.
30000
 <NEXT_NE> Major General Chun Doo <NEXT_NE> December 1979 <NEXT_NE> Chun Doo-hwan <NEXT_NE> Choi <SENT_START> In

In [ ]:
train_df

,index,complex,simple,complex_NER,simple_NER,co_occur_NER
0,0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, as late as 1814, Anna, 1...","[James Edward, Anna Austen, 1814]",[]
1,1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Mandaean, Mani, Säve-Söderb..."
2,2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, Persephone, Hermes, Hades...","[Underworld, Persephone, Demeter, Hades]","[Persephone, Hades]"
3,3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[]
...,...,...,...,...,...,...
249161,296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
249162,296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[France, Saint-Maixant]"
249163,296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[]
249164,296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe, one]","[one, Boulogne-Billancourt, Europe]"


## dataset that contains NE appear in both sides 

In [ ]:
  def substringSieve(NER_list,re_NER):
    string_list = []
    string_list.extend(NER_list)
    string_list.extend(re_NER)
    
    string_list.sort(key=lambda s: len(s), reverse=True)
    out = []
    for s in string_list:
        if not any([s in o for o in out]):
            out.append(s)
    return out

  def lcs(S,T):
    '''
    From https://www.bogotobogo.com/python/python_longest_common_substring_lcs_algorithm_generalized_suffix_tree.php
    '''
    m = len(S)
    n = len(T)
    counter = [[0]*(n+1) for x in range(m+1)]
    longest = 0
    lcs_set = set()
    for i in range(m):
        for j in range(n):
            if S[i] == T[j]:
                c = counter[i][j] + 1
                counter[i+1][j+1] = c
                if c > longest:
                    lcs_set = set()
                    longest = c
                    lcs_set.add(S[i-c+1:i+1])
                elif c == longest:
                    lcs_set.add(S[i-c+1:i+1])
    
    if len(lcs_set) == 1:# and ' ' not in list(lcs_set)[0]:
        return list(lcs_set)[0]
    else:
        return ''


In [ ]:
def get_NE_sent_intersect(a,simple_sent):
  # get the intersection of NE list and sentence
  output = []
  for index,item in enumerate(a):
    tmp_str = lcs(item,simple_sent)
    lower_tmp_str = lcs(item.lower(),simple_sent.lower())

    # print('| NE: item, tmp_str, lower_tmp_str\n')
    # print(item)
    # print(tmp_str)
    # print(lower_tmp_str)

    if tmp_str != '' and lower_tmp_str != '':

      if tmp_str[0] ==' ':
        tmp_str = tmp_str[1:]
        # print('modified tmp_str',tmp_str)

      if lower_tmp_str[0] ==' ':
        lower_tmp_str = lower_tmp_str[1:]
        # print('modified lower_tmp_str',lower_tmp_str)
    
      if tmp_str != '' and tmp_str[-1] ==' ':
        tmp_str = tmp_str[:-1]
        # print('modified tmp_str',tmp_str)
      if lower_tmp_str != '' and lower_tmp_str[-1] ==' ':
        lower_tmp_str = lower_tmp_str[:-1]
        # print('modified lower_tmp_str',lower_tmp_str)

      if len(tmp_str) != 0:
        if len(lower_tmp_str) == len(tmp_str)+1:
          # lower case NE exist in smiple sentence
          output.append(lower_tmp_str)
          # print('output',output)
        else:
          output.append(tmp_str)
          # print('output',output)

    # print('get_NE_sent_intersect output',output)
  
  return output


In [ ]:
def get_rid_of_subword(a,interset_list):
  # check if intersection is not a complete word in its original NE list
  output = []

  NE_lists = [x.split(' ') for x in a]
  list_lists = [item for sublist in NE_lists for item in sublist]
  
  # print('NE_lists',NE_lists)
  # print('list_lists',list_lists)

  for index,interset_item in enumerate(interset_list):
    # print('index,interset_item',index,interset_item)

    if any(x in interset_item for x in list_lists) or any(x.lower() in interset_item for x in list_lists):
      output.append(interset_item)
      # print('get_rid_of_subword output',output)

  return output


In [ ]:
stop_words_list = ["a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]
cap_stop_words_list = [x.title() for x in stop_words_list]
stop_words_all = []
stop_words_all.extend(stop_words_list)
stop_words_all.extend(cap_stop_words_list)
len(stop_words_all)


1568

In [ ]:
df_name = ['train_df','valid_df','test_df']

train_df

,index,complex,simple,complex_NER,simple_NER,co_occur_NER
0,0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, as late as 1814, Anna, 1...","[James Edward, Anna Austen, 1814]",[]
1,1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Mandaean, Mani, Säve-Söderb..."
2,2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, Persephone, Hermes, Hades...","[Underworld, Persephone, Demeter, Hades]","[Persephone, Hades]"
3,3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[]
...,...,...,...,...,...,...
249161,296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
249162,296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[France, Saint-Maixant]"
249163,296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[]
249164,296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe, one]","[one, Boulogne-Billancourt, Europe]"


In [ ]:
complex_ratio,simple_ratio,intersect_NE = [],[],[]

for i in range(3):   ### Because there are three training datasets in the df_names
  tmp_complex_ratio = []
  tmp_simple_ratio = []
  tmp_NE = []

  for j in range(len(globals()[df_name[i]])):
    
    if j % 10000 == 0:
      print(i,j)
    # print(i,j)
    a = globals()[df_name[i]].complex_NER[j].copy()
    b = globals()[df_name[i]].simple_NER[j].copy()

    complex_sent = globals()[df_name[i]].complex[j]
    simple_sent = globals()[df_name[i]].simple[j]
  
    # print(a,b)
    # print(complex_sent,'\n',simple_sent)

    # get the intersection of NE list and sentence
    complex_NE_intersect_simple = get_NE_sent_intersect(a,simple_sent)
    simple_NE_intersect_complex = get_NE_sent_intersect(b,complex_sent)

    # delete the intersection if is not a complete word in its original NE list
    # print('a,complex_NE_intersect_simple',a,complex_NE_intersect_simple)
    complex_NE_intersect_simple_final = get_rid_of_subword(a,complex_NE_intersect_simple)
    # print('b,simple_NE_intersect_complex',b,simple_NE_intersect_complex)
    simple_NE_intersect_complex_final = get_rid_of_subword(b,simple_NE_intersect_complex)

    # merge two lists into one and delete substring of any existing string
    output = substringSieve(complex_NE_intersect_simple_final,simple_NE_intersect_complex_final)
    # print('output',output)

    # delete stop words and capital stop words
    final = []
    for index,item in enumerate([x.split(' ') for x in output]):
      # if the item is 'WORD ' or ' WORD'

      # print('item,len(item)',item,len(item))
      if len(item) == 2 and '' in item:
        space_index = item.index('')
        str_index = 1 - space_index
        # dont append the word if it is a stop word
        if item[str_index] not in stop_words_all:
          final.append(item[str_index])
          # print('item[str_index]',item[str_index])

      # item is a phrase 
      else:
        clean_item = ' '.join(item)
        if clean_item[-1] == ' ':
          clean_item = clean_item[:-1]
        # print('clean_item',clean_item)
        final.append(clean_item)

    # print('final',final)
    
    # break down NE in complex_NE and simple_NE list to check if any NE is not included in the final output due to some special case such as
    '''
    complex sentence: In Holland they were called Stadspijpers, in Germany Stadtpfeifer and in Italy Pifferi.	
    simple sentence: They were called Stadtpfeifer in Germany and Pifferi in Italy.	
    NE in complex sentence: [Germany Stadtpfeifer, Italy Pifferi, Stadspijpers, Holland]
    NE in simple sentence: [Stadtpfeifer in Germany, Pifferi in Italy]
    
    the function lcs only look for the longest common substring, so when do lcs('Germany Stadtpfeifer',simple_sentence), the output is Stadtpfeifer.
    And when do lcs('Stadtpfeifer in Germany',complex_sentence), the output is also Stadtpfeifer.
    Hence 'Germany' is ignored in both cases.
    '''

    # split the item and flat the list of lists
    c = [item for sublist in [x.split(' ') for x in a] for item in sublist] # e.g.  ['Germany', 'Stadtpfeifer', 'Italy', 'Pifferi', 'Stadspijpers', 'Holland']
    d = [item for sublist in [x.split(' ') for x in b] for item in sublist] # e.g. ['Stadtpfeifer', 'in', 'Germany', 'Pifferi', 'in', 'Italy']
    
    intersection_1 = [x for ele in c for x in d if x in ele] # element in d that also is a (sub)string in c
    intersection_2 = [x for ele in d for x in c if x in ele]

    # do substringSieve twice
    final_output = substringSieve(substringSieve(intersection_1,intersection_2),final)
    # print('final_output',final_output)

    FINAL = [element for element in final_output if element not in stop_words_all]
    
    # print('FINAL',FINAL)
    ####################
    tmp_NE.append(FINAL)
    ####################

    # find the ratio of len(complex_NE_in_both)/len(complex) and the same for simple
    if len(FINAL) != 0:

      cc = list(set([element for element in c if element not in stop_words_all]))
      dd = list(set([element for element in d if element not in stop_words_all]))

      complex_in_inter_num = sum([x in FINAL for x in cc])
      # print('complex_in_inter_num',complex_in_inter_num)

      simple_in_inter_num = sum([x in FINAL for x in dd])
      # print('simple_in_inter_num',simple_in_inter_num)

      if len(a) != 0:
        complex_in_inter_ratio = complex_in_inter_num/len(a)
      else:
        complex_in_inter_ratio = np.nan

      if len(b) != 0:
        simple_in_inter_ratio = simple_in_inter_num/len(b)
      else:
        simple_in_inter_ratio = np.nan

      # print(complex_in_inter_ratio,simple_in_inter_ratio)
      ####################
      tmp_complex_ratio.append(complex_in_inter_ratio)
      tmp_simple_ratio.append(simple_in_inter_ratio)
      ####################

  complex_ratio += [tmp_complex_ratio]
  simple_ratio += [tmp_simple_ratio]
  intersect_NE += [tmp_NE]
    
  

0 0
0 10000
0 20000
0 30000
0 40000
0 50000
0 60000
0 70000
0 80000
0 90000
0 100000
0 110000
0 120000
0 130000
0 140000
0 150000
0 160000
0 170000
0 180000
0 190000
0 200000
0 210000
0 220000
0 230000
0 240000
1 0
2 0


In [ ]:
# while True:pass

KeyboardInterrupt: ignored

In [ ]:
intersect_NE[0][0:10]

In [ ]:
with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/NER_cooc_0823', 'wb') as fp:
    pickle.dump(intersect_NE, fp)

with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/complex_ratio', 'wb') as fp:
    pickle.dump(complex_ratio, fp)

with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/simple_ratio', 'wb') as fp:
    pickle.dump(simple_ratio, fp)

In [ ]:
with open ('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/NER_cooc_0823', 'rb') as fp:
    NER_cooc_0823 = pickle.load(fp)

In [ ]:
# valid_df

In [ ]:
# while True:pass

In [ ]:
train_df['co_occur_NER'] = NER_cooc_0823[0]
valid_df['co_occur_NER'] = NER_cooc_0823[1]
test_df['co_occur_NER'] = NER_cooc_0823[2]

In [ ]:
test_df

,complex,simple,complex_NER,simple_NER,co_occur_NER
0,One side of the armed conflicts is composed ma...,One side of the armed conflicts is made of Sud...,"[Afro-Arab Abbala, Janjaweed, Sudanese, Rizeigat]","[Afro-Arab Abbala, Janjaweed, Sudanese, Rizeigat]","[Afro-Arab Abbala, Janjaweed, Sudanese, Rizeigat]"
1,"Jeddah is the principal gateway to Mecca, Isla...","Jeddah is the main gateway to Mecca, Islam's h...","[Muslims, Jeddah, Mecca, Islam]","[Muslims, Jeddah, Mecca, Islam]","[Muslims, Jeddah, Mecca, Islam]"
2,The Great Dark Spot is thought to represent a ...,The Great Dark Spot is thought to represent a ...,"[Great Dark Spot, Neptune]","[Great Dark Spot, Neptune]","[Great Dark Spot, Neptune]"
3,"His next work, Saturday, follows an especially...",His next work at Saturday will be a successful...,[Saturday],[Saturday],[Saturday]
4,"The tarantula, the trickster character, spun a...","The tarantuala, the trickster, spun a black ch...",[],[tarantuala],[]
...,...,...,...,...,...
354,Although the name suggests that they are locat...,Eventhough the name evokes that they are place...,"[Bernese Oberland, Bernese Alps, Obwalden, Fri...","[Bernese Oberland, Eventhough the, Bernese Alp...","[Bernese Oberland, Bernese Alps, though the, O..."
355,"There he had one daughter, later baptized as M...",He had one daughter named Mary Ann Fisher Powe...,"[Mary Ann Fisher Power, one]","[Mary Ann Fisher Power, one]",[Mary Ann Fisher Power]
356,"During an interview, Edward Gorey mentioned th...","During an interview, Edward Gorey said that Ba...","[Edward Gorey, Bawden]","[Edward Gorey, Bawden]","[Edward Gorey, Bawden]"
357,The string can vibrate in different modes just...,The string can vibrate in different modes just...,[],[],[]


In [ ]:
with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/train_df_0911', 'wb') as fp:
    pickle.dump(train_df, fp)

with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/valid_df_0911', 'wb') as fp:
    pickle.dump(valid_df, fp)

with open('/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/test_df_0911', 'wb') as fp:
    pickle.dump(test_df, fp)

create dataset(source) 

In [ ]:
target_dir = '/content/drive/MyDrive/Baseline_MUSS/muss/resources/datasets/wikilarge_cooc_0823'
!mkdir $target_dir

In [ ]:
data_names = ['train_complex', 'valid_complex', 'test_complex']
df_list = ['train_df','valid_df','test_df']

for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        # print(sentence)
        NER = row.co_occur_NER

        sentence_w_control_token = " <NEXT_NE> " +" <NEXT_NE> ".join(NER) + " <SENT_START> " + sentence

        if index%10000==0:
            print(index)
            print(sentence_w_control_token)

        f.write("{}\n".format(sentence_w_control_token))


  f.close()


0
 <NEXT_NE> James Edward <NEXT_NE> Austen <NEXT_NE> s late <NEXT_NE> 1814 <NEXT_NE> Anna <SENT_START> There is manuscript evidence that Austen continued to work on these pieces as late as the period 1809 â "11, and that her niece and nephew, Anna and James Edward Austen, made further additions as late as 1814.
10000
 <NEXT_NE> of Lebanon <NEXT_NE> 1992 <SENT_START> In 1992, he became a prime minister, under Syria's watchful and military occupation of Lebanon, he put the country back on the financial map through the issuing of Eurobonds and won plaudits from the World Bank for his plan to borrow reconstruction money as the country's debt grew to become the largest per capita in the world.
20000
 <NEXT_NE> Varanasi <NEXT_NE> Lucknow <NEXT_NE> Kanpur <NEXT_NE> Agra <SENT_START> Domestic airports are Lucknow, GorakhpurAgra, Kanpur Allahabad and Varanasi.
30000
 <NEXT_NE> Major General Chun Doo <NEXT_NE> December 1979 <NEXT_NE> Chun Doo-hwan <NEXT_NE> Choi <SENT_START> In December 1979, Ma

## create training set (target sentence)

In [ ]:
target_dir = '/content/drive/MyDrive/Baseline_MUSS/muss/resources/datasets/wikilarge_final_simple_0821' ### I've made changes here to change to correct directory (Yijie)

In [ ]:
data_names = ['train_simple', 'valid_simple', 'test_simple']
df_list = ['train_df','valid_df','test_df']
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.simple

        if index%10000==0:
            print(index)
            print(sentence)

        f.write("{}\n".format(sentence))

        # break

  f.close()


0
There is some proof that Austen continued to work on these pieces later in life. Her nephew and niece, James Edward and Anna Austen, may have made further additions to her work in around 1814.
10000
He was also the Prime Minister of Lebanon from 1992 to 1998, and he became Prime Minister again in 2000.
20000
They are in Agra, Kanpur, Lucknow and Varanasi.
30000
In December 1979, Major General Chun Doo-hwan and close allies in the military staged a military coup against Choi's government.
40000
To make it possible to read faster, certain transcriptions of Braille use contractions.
50000
The Fairly OddParents became a huge hit, second only in the ratings to SpongeBob SquarePants.
60000
Christians believe that through faith in God they are spiritually made alive with Jesus so that they may lead a new life.
70000
Bartók and Kodály travelled together into the countryside to collect and study old Hungarian folk melodies.
80000
It and its compounds are used mainly in science research though

# operation lexical simplification

In [ ]:
train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS/muss/qualitative/test_df_0911")

Training set
obtain CERF level using Amplifi engine

In [ ]:
import requests
import json
import ast
from multiprocessing import Pool


In [ ]:
# API
url = #

headers = {
    'Content-Type': 'application/json',
    'x-api-key' : #
}

SyntaxError: ignored

In [ ]:
while True:pass

KeyboardInterrupt: ignored

In [ ]:
train_df.drop()

,index,complex,simple,complex_NER,simple_NER,co_occur_NER
0,0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, as late as 1814, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, s late, 1814, Anna]"
1,1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M...","[Psalms of Thomas, Säve-Söderberg, Mandaean, M..."
2,2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, Persephone, Hermes, Hades...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]"
3,3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration]
4,4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[]
...,...,...,...,...,...,...
249161,296397,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok]
249162,296398,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i..."
249163,296399,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk]
249164,296400,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe, one]","[Boulogne-Billancourt, Europe]"


In [4]:
train_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/train_df_0911")
valid_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/valid_df_0911")
test_df = pd.read_pickle("/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/test_df_0911")

In [10]:
with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/ABCD_train_C2C1B2_0911', 'rb') as f:
    train_C2C1B2_df = pickle.load(f)

with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/ABCD_valid_C2C1B2_0911', 'rb') as f:
    valid_C2C1B2_df = pickle.load(f)


# with open('/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/ABCD_test_C2C1B2_0911', 'rb') as f:
#     test_C2C1B2_df = pickle.load(f)


In [6]:
TRAIN_DF = pd.concat([train_df.drop(columns=['level_0','index']),train_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
TRAIN_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,There is manuscript evidence that Austen conti...,There is some proof that Austen continued to w...,"[James Edward Austen, the period 1809, Anna, 1...","[James Edward, Anna Austen, 1814]","[James Edward, Austen, Anna, 1814]","[manuscript, evidence]"
1,"In a remarkable comparative analysis, Mandaean...",Mandaean scholar Säve-Söderberg showed that Ma...,"[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[Psalms of Thomas, Säve-Söderberg, Mani's Psal...","[comparative, analysis]"
2,"Before Persephone was released to Hermes, who ...",When Demeter went to the Underworld to rescue ...,"[a period each year, six or three, Persephone,...","[Underworld, Persephone, Demeter, Hades]","[Persephone, underworld, Hades]","[released, tricked]"
3,Cogeneration plants are commonly found in dist...,Cogeneration plants are commonly found in dist...,[Cogeneration],[Cogeneration],[Cogeneration],NaN
4,"Geneva (,;,;,;;) is the second-most-populous c...",The city's main newspaper is the Tribune de Ge...,"[Switzerland, Romandie, Geneva, second, Zürich...","[the Tribune de Genève, 1 February 1879, James...",[city],"[populous, populous]"
...,...,...,...,...,...,...
249161,These structures form at the high-pressure dep...,These structures form at the high-pressure dep...,[Lake Vostok],[Lake Vostok],[Lake Vostok],NaN
249162,Saint-Maixant is a commune in the Gironde depa...,"Saint-Maixant, Gironde is a commune. It is fou...","[Saint-Maixant, Aquitaine in, Gironde, France]","[Aquitaine in the Gironde, Saint-Maixant, France]","[ne in the Gironde, Saint-Maixant, Aquitaine i...",NaN
249163,The Hollywood Walk of Fame is a series of side...,There are many stars on the sidewalk.,"[Vine Street in Hollywood, Hollywood Walk of F...",[],[walk],"[sidewalks, monument, achievement]"
249164,Boulogne-Billancourt is the most populous subu...,"Consequently, Boulogne-Billancourt is one of t...","[Boulogne-Billancourt, Europe, Paris, one]","[Boulogne-Billancourt, Europe]","[Boulogne-Billancourt, Europe]",[populous]


In [7]:
VALID_DF = pd.concat([valid_df.drop(columns=['level_0','index']),valid_C2C1B2_df.drop(columns=['complex','simple','complex_NER','simple_NER','co_occur_NER','complex_read_level','simple_read_level'])], axis=1)
VALID_DF

,complex,simple,complex_NER,simple_NER,co_occur_NER,C2C1B2_in_complex
0,Upper Sorbian is a minority language spoken by...,"There are around 40,000 speakers of Upper Sorb...","[Sorbs in Germany in the, Upper Sorbian, Upper...",[the European Charter for Regional or Minority...,"[Upper Sorbian, in Germany, inority, Saxony]",[ica]
1,It used to be on display as part of the Americ...,It is on permanent display as part of the Amer...,"[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...","[Library of Congress in Washington, American T...",NaN
2,Relation to the Nobel Prize The Sveriges Riksb...,The Sveriges Riksbank Prize in Economic Scienc...,[Nobel Prize The Sveriges Riksbank Prize in Ec...,[Sveriges Riksbank Prize in Economic Sciences ...,[The Sveriges Riksbank Prize in Economic Scien...,NaN
3,A shoe is an item of footwear evolved at first...,A shoe is also an item of clothing.,[first],[],[],[evolved]
4,Of the eleven tropical cyclones that formed du...,"The season officially started on May 15, 1995 ...","[Hurricane Ismael, at least 116, the season, e...","[the eastern Pacific, November 30, 1995, May 1...","[the season, the e]","[tropical, cyclones, affected]"
...,...,...,...,...,...,...
812,Sedimentary rock is one of the three main rock...,Other types of rock are igneous rock and metam...,"[Sedimentary, three, one]",[],[],NaN
813,"""It took more than 60 years after 1859 until t...","""It took more than 60 years after 1859 until t...",[more than 60 years after 1859],[more than 60 years after 1859],[more than 60 years after 1859],NaN
814,"East Timor, also known as Timor-Leste (Tetum: ...",The Democratic Republic of East Timor or Timor...,"[the Democratic Republic of Timor-Leste, South...","[Democratic Republic of East Timor, Southeast ...","[the democratic republic of, Democratic Republ...",[officially]
815,Certain artistic traditions that originated in...,Certain traditions that originated in the Byza...,"[the Byzantine Empire, Eastern Orthodox, the p...","[the Byzantine Empire, Eastern Orthodox, Greec...","[the Byzantine Empire, Eastern Orthodox, Greec...",NaN


In [11]:
target_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/0913_ABCD_NER_wikilarge'

In [12]:
data_names = ['train_complex', 'valid_complex']
df_list = ['TRAIN_DF','VALID_DF']

In [13]:
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')

  
  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.complex
        NER = row.simple_NER
        hard_word = row.C2C1B2_in_complex

        if not isinstance(hard_word, float):

          sentence_w_control_token = " <NEXT_NE> "+" <NEXT_NE> ".join(NER)+" <NEXT_DIFFICULT_WORD> "+" <NEXT_DIFFICULT_WORD> ".join(hard_word)+ " <SENT_START> " + sentence


          if index%10000==0:
              print(index)
              print(sentence_w_control_token)

          f.write("{}\n".format(sentence_w_control_token))

        # break

  f.close()

FileNotFoundError: ignored

In [ ]:
data_names = ['train_simple', 'valid_simple']
# train.simple & valid.simple
for index,data_name in enumerate(data_names):

  chosen_df = globals()[df_list[index]]
  file_name = data_name.replace('_','.')
  
  with open(target_dir+'/'+file_name, 'w') as f:

    for index, row in chosen_df.iterrows():

        sentence = row.simple
        hard_word = row.C2C1B2_in_complex

        if not isinstance(hard_word, float):

          if index%10000==0:
              print(index)
              print(sentence)

          f.write("{}\n".format(sentence))

        # break

  f.close()